# Model Analysis

In [302]:
import numpy as np
import glob
import warnings
import random
from pathlib import Path
from gensim.models.word2vec import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# %matplotlib notebook (makes plot interactive but slows things down and makes them buggy eventually)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

In [303]:
warnings.filterwarnings("ignore")

model_dir = Path("model/")
model_files = glob.glob(str(model_dir / "*.model"))

In [304]:
# Get models by their split date
model_files_griner = [model_file for model_file in model_files if "07_Oct" in model_file]
model_files_twitter = [model_file for model_file in model_files if "01_Oct" in model_file]
model_files_pelosi = [model_file for model_file in model_files if "26_Oct" in model_file]
model_files_colorado_springs = [model_file for model_file in model_files if "18_Nov" in model_file]

In [305]:
model_files_griner

['model/01_Jun_to_07_Oct.model', 'model/07_Oct_to_28_Apr.model']

In [306]:
model_files_twitter

['model/01_Oct_to_28_Apr.model', 'model/01_Jun_to_01_Oct.model']

In [307]:
model_files_pelosi

['model/26_Oct_to_28_Apr.model', 'model/01_Jun_to_26_Oct.model']

In [308]:
model_files_colorado_springs

['model/01_Jun_to_18_Nov.model', 'model/18_Nov_to_28_Apr.model']

In [309]:
model_griner_pre = Word2Vec.load(model_files_griner[0])
model_griner_post = Word2Vec.load(model_files_griner[1])

model_twitter_pre = Word2Vec.load(model_files_twitter[1])
model_twitter_post = Word2Vec.load(model_files_twitter[0])

model_pelosi_pre = Word2Vec.load(model_files_pelosi[1])
model_pelosi_post = Word2Vec.load(model_files_pelosi[0])

model_colorado_springs_pre = Word2Vec.load(model_files_colorado_springs[0])
model_colorado_springs_post = Word2Vec.load(model_files_colorado_springs[1])

model_static = Word2Vec.load("model/word2vec.model")

# 2-D Visualization

2D visualization of word-embeddings.
Function can use either pca or tsne to reduce dimensions of the word vectors.
Function can either use matplotlib or plotly for plotting

In [165]:
def display_scatterplot(model, words, keyword, title = "Words in the embedding space", subtitle = None, use_tsne=False, use_plotly = False):

    # we do not want to change mutable parameters
    word_list = words.copy()

    # keywords not in words will otherwise not get visualized
    if keyword not in word_list:
        word_list.append(keyword)

    # Iff use_tsne is true, perform t-SNE, otherwise perform PCA
    if use_tsne:
        vectors_2d = perform_tsne(model.wv, word_list)
    else:
        vectors_2d = perform_pca(model.wv, word_list)

    # Iff use_plotly is true, use plotly for plotting, otherwise matplotlib
    if use_plotly:
        plot_with_plotly(vectors_2d, word_list, keyword, title = title, subtitle = subtitle)
    else:
        plot_with_matplotlib(vectors_2d, word_list, keyword, title = title, subtitle = subtitle)

## Variants of Plotting

In [229]:
def plot_with_plotly(vectors, labels, keyword, title = "Words in the embedding space", subtitle = None, plot_in_notebook=True):

    # Create a list to hold scatter plot data points
    data = []
    text_offset = 0.05  # Offset for positioning text labels

    # Iterate through each vector and label to create scatter plot data
    for i, (x, y) in enumerate(vectors):
        # keyword is red, other words are blue
        text_color = 'red' if labels[i] == keyword else 'black'
        # Append a scatter plot trace for the current label
        data.append(go.Scatter(x=[x], y=[y], mode='markers+text', text=[labels[i]], textfont=dict(color=text_color),
                               marker=dict(color=text_color), textposition='top center'))

    # Define the layout settings for the plot
    layout = go.Layout(
        title=title,
        titlefont=dict(size=22),
        title_x=0.5, # Center the title horizontally
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        showlegend = False,
        annotations=[
            dict(
                x=0.5,
                y=1.08,
                xref='paper',
                yref='paper',
                text=subtitle,
                showarrow=False,
            )
        ],
    )

    # Create a Figure object with the data and layout
    fig = go.Figure(data=data, layout=layout)

    # Display the plot in a notebook or save to an HTML file
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(fig, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [230]:
def plot_with_matplotlib(vectors, labels, keyword, title = "Words in the embedding space", subtitle = None):

    # Create a scatter plot
    plt.figure(figsize=(6,6))
    plt.scatter(vectors[:,0], vectors[:,1], edgecolors='k', c='orange')

    # Add word labels to the scatter plot
    for word, (x,y) in zip(labels, vectors):
        color = 'red' if word == keyword else 'orange'
        plt.text(x + 0.02, y + 0.02, word, color=color)

    # Set plot titles
    plt.suptitle(title, fontsize=14)
    if subtitle:
        plt.title(subtitle, fontsize=10)

## Variants of dimension reduction

In [231]:
def perform_pca(model, words):

    # Convert the word vectors of the specified words into a NumPy array

    word_vectors = np.array([model[w] for w in words])

    # 1. Apply PCA to reduce dimensionality to 2 components
    # 2. Perform the PCA transformation on the word vectors and select the first 2 dimensions
    return PCA().fit_transform(word_vectors)[:, :2]

In [232]:
from sklearn.preprocessing import StandardScaler
def perform_tsne(model, words):

    # Convert the word vectors of the specified words into a NumPy array
    word_vectors = np.array([model[w] for w in words])

    # perplexity must not exceed the number of samples
    perplexity = min(5, len(words) - 1)

    # 1. Apply t-SNE with 2 components, using perplexity adjusted based on the number of words
    # 2. Perform the t-SNE transformation on the word vectors
    tsne_vectors = TSNE(n_components=2, perplexity=perplexity, random_state=1040).fit_transform(word_vectors)
    # Fit the scaler on t-SNE vectors
    scaler = StandardScaler()
    tsne_vectors = scaler.fit_transform(tsne_vectors)

    return tsne_vectors

## Sampling methods

In [233]:
def sample_from_most_similar(model_pre, model_post, keyword, sample_size=6):

    # we only calculate the 100 most similar words
    sample_size = min(20, sample_size)

    similar_words_pre = model_pre.wv.most_similar(keyword, topn=20)
    similar_words_post = model_post.wv.most_similar(keyword, topn=20)

    sampled_words_pre = random.sample(similar_words_pre, sample_size // 2)
    sampled_words_post = random.sample(similar_words_post, sample_size // 2)
    sampled_words = sampled_words_pre + sampled_words_post

    return [key for key,_ in sampled_words]

# Experiment

## Release of Brittney Griner

On December 8, 2022, Russia and the United States conducted a prisoner exchange, trading Brittney Griner, an American basketball player, for Viktor Bout, a Russian arms dealer. Griner, a WNBA champion star and Team USA Olympic athlete, had been convicted of smuggling and possession of cannabis in Russia earlier in 2022 and sentenced to nine years in prison.

Source: [Wikipedia article](https://en.wikipedia.org/wiki/Viktor_Bout%E2%80%93Brittney_Griner_prisoner_exchange)

In [234]:
test_word_griner = "griner"

01.06.2022 - 07.10.2022

In [316]:
model_griner_pre.wv.most_similar(test_word_griner)

[('brittney', 0.9746803045272827),
 ('preacher', 0.934641420841217),
 ('whelan', 0.9327844977378845),
 ('pnjaban', 0.9199792742729187),
 ('bombing', 0.9166041016578674),
 ('leblanc', 0.9103859663009644),
 ('freedman', 0.9102946519851685),
 ('bongino', 0.9100096821784973),
 ('mcdowell', 0.9095998406410217),
 ('stooges', 0.9077529907226562)]

07.10.2022 - 28.04.2023

In [318]:
model_griner_post.wv.most_similar(test_word_griner)

[('zelensky', 0.8480092287063599),
 ('cornyn', 0.8175973296165466),
 ('kremlin', 0.8124685287475586),
 ('bannons', 0.8099071383476257),
 ('mccausland', 0.8074005842208862),
 ('mcfadden', 0.7868133783340454),
 ('abe', 0.7855336666107178),
 ('volodymyr', 0.7850770950317383),
 ('fervently', 0.7821239829063416),
 ('ronnyjacksontx', 0.7781116366386414)]

In [319]:
vis_list_griner = ["brittney", "freedman", "32yearold", "kremlin", "zelensky", "vladimir", "joni"]
title_griner = "Brittney Griner prisoner exchange"

In [320]:
display_scatterplot(model_griner_pre, vis_list_griner, test_word_griner, title=title_griner, subtitle="01 June 2022 - 07 October 2022", use_tsne=False, use_plotly=True)

In [321]:
display_scatterplot(model_griner_post, vis_list_griner, test_word_griner, title=title_griner, subtitle="07 October 2022 - 28 April 2023", use_tsne=False, use_plotly=True)

## Elon Musk Twitter takeover

Business magnate Elon Musk initiated an acquisition of American social media company Twitter, Inc. on April 14, 2022, and concluded it on October 27, 2022. Musk had begun buying shares of the company in January 2022, becoming its largest shareholder by April with a 9.1 percent ownership stake

source: [Wikipedia article](https://en.wikipedia.org/wiki/Acquisition_of_Twitter_by_Elon_Musk#:~:text=Business%20magnate%20Elon%20Musk%20initiated,a%209.1%20percent%20ownership%20stake.)

In [322]:
test_word_twitter = "musk"

In [323]:
model_twitter_pre.wv.most_similar(test_word_twitter)

[('elon', 0.8883422613143921),
 ('breakfastclubam', 0.7354350090026855),
 ('socialmedia', 0.7335272431373596),
 ('gab', 0.7229589819908142),
 ('referencing', 0.7129106521606445),
 ('mitchs', 0.7111585736274719),
 ('bezos', 0.7030009627342224),
 ('condemnation', 0.7000606656074524),
 ('doubting', 0.6998190879821777),
 ('musks', 0.694724977016449)]

In [324]:
model_twitter_post.wv.most_similar(test_word_twitter)

[('elon', 0.9781531095504761),
 ('musks', 0.950725257396698),
 ('amandamarcotte', 0.9251418113708496),
 ('mattwallace888', 0.9056128859519958),
 ('twitter', 0.8492724299430847),
 ('followers', 0.7500432133674622),
 ('tweets', 0.6832248568534851),
 ('troll', 0.6627514958381653),
 ('screenshot', 0.6527742743492126),
 ('roth', 0.6525090932846069)]

In [325]:
vis_list_twitter = ["facebook", "instagram", "tiktok", "user", "twitter", "elon", "musk", "tesla"]
title_twitter = "Acquisition of Twitter by Elon Musk"

In [326]:
display_scatterplot(model_twitter_pre, vis_list_twitter, test_word_twitter,title_twitter, "01 June 2022 - 01 October 2022", use_tsne=False, use_plotly=True)

In [327]:
display_scatterplot(model_twitter_post, vis_list_twitter, test_word_twitter, title_twitter, "01 October 2022 - 28 April 2023", use_tsne=False, use_plotly=True)

## Attack on Paul Pelosi

On October 28, 2022, an intruder attacked Paul Pelosi, the 82-year-old husband of Nancy Pelosi, then the Speaker of the United States House of Representatives. The assailant beat Paul Pelosi with a hammer during a home invasion burglary of the couple's residence in Pacific Heights, San Francisco. He was seriously injured and underwent surgery for his fractured skull.

source: [Wikipedia article](https://en.wikipedia.org/wiki/Attack_on_Paul_Pelosi)

In [328]:
test_word_pelosi = "hammer"

In [329]:
model_pelosi_pre.wv.most_similar(test_word_pelosi)

[('nieces', 0.7448627948760986),
 ('tylenol', 0.7412106394767761),
 ('licking', 0.7287212610244751),
 ('wrist', 0.7271592617034912),
 ('numb', 0.7244389653205872),
 ('devastated', 0.7229718565940857),
 ('meds', 0.7218886613845825),
 ('blackouts', 0.7192128300666809),
 ('nerves', 0.7114683985710144),
 ('resorted', 0.7101452946662903)]

In [332]:
model_pelosi_post.wv.most_similar(test_word_pelosi)

[('pelosis', 0.881808876991272),
 ('nancy', 0.7820796370506287),
 ('fractured', 0.7476246356964111),
 ('pelosi', 0.7451282739639282),
 ('rapper', 0.728460967540741),
 ('intruder', 0.7266745567321777),
 ('attacked', 0.7140253782272339),
 ('moscow', 0.7115112543106079),
 ('mitchs', 0.7076058387756348),
 ('sfsu', 0.7065549492835999)]

In [333]:
vis_list_pelosi = ["hammer", "nail", "carpenter", "pelosi", "fractured", "pelosis"]
title_pelosi = "Attack on Paul Pelosi"

In [334]:
display_scatterplot(model_pelosi_pre, vis_list_pelosi, test_word_pelosi, title_pelosi,
                    "01 June 2022 - 26 October 2022", use_tsne=False, use_plotly=True)

In [335]:
display_scatterplot(model_pelosi_post, vis_list_pelosi, test_word_pelosi, title_pelosi, "26 October 2022 - 28 April 2023", use_tsne=False, use_plotly=True)

## Colorado Springs shooting at LGBTQ nightclub

On November 19–20, 2022, an anti-LGBT-motivated mass shooting occurred at Club Q, a gay bar in Colorado Springs, Colorado, United States. Five people were murdered, and 25 others were injured, 19 of them by gunfire. The shooter, 22-year-old Anderson Lee Aldrich, was also injured while being restrained, and was taken to a local hospital

source: [wikipedia article](https://en.wikipedia.org/wiki/Colorado_Springs_nightclub_shooting)

In [336]:
test_word_colorado_springs = "nightclub"

In [337]:
model_colorado_springs_pre.wv.most_similar(test_word_colorado_springs)

[('suicides', 0.7964819669723511),
 ('ebola', 0.7863858938217163),
 ('shenzhen', 0.7564419507980347),
 ('floods', 0.7336622476577759),
 ('wildfire', 0.7312685251235962),
 ('shanghai', 0.7243152260780334),
 ('viral', 0.7237164378166199),
 ('twomonth', 0.721132755279541),
 ('waves', 0.7191981077194214),
 ('indications', 0.7128860950469971)]

In [338]:
model_colorado_springs_post.wv.most_similar(test_word_colorado_springs)

[('antitrump', 0.8626054525375366),
 ('murders', 0.8326581716537476),
 ('thepostmillennialcomtransmalearr', 0.827488899230957),
 ('spree', 0.8128408789634705),
 ('deadliest', 0.8057740330696106),
 ('mass', 0.8000726103782654),
 ('colorado', 0.7880188226699829),
 ('gunman', 0.7809202671051025),
 ('rampage', 0.7794814109802246),
 ('springs', 0.7699750065803528)]

In [339]:
vis_list_colorado_springs = ["kentucky", "baltimore", "denver", "massacre", "gunman", "nightclub", "springs"]
title_colorado_springs = "Colorado Springs nightclub shooting"

In [340]:
display_scatterplot(model_colorado_springs_pre, vis_list_colorado_springs, test_word_colorado_springs, title_colorado_springs, "01 June 2022 - 18 November 2022", use_tsne=False, use_plotly=True)

In [341]:
display_scatterplot(model_colorado_springs_post, vis_list_colorado_springs, test_word_colorado_springs, title_colorado_springs, "18 November 2022 - 28 April 2023", use_tsne=False, use_plotly=True)